clone and cd into repo

In [ ]:
# first download the gpt-2 code
!git clone https://github.com/nshepperd/gpt-2.git

Cloning into 'gpt-2'...
remote: Enumerating objects: 435, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 435 (delta 19), reused 48 (delta 13), pack-reused 371
Receiving objects: 100% (435/435), 4.48 MiB | 12.29 MiB/s, done.
Resolving deltas: 100% (220/220), done.


In [ ]:
!pwd

/content


In [ ]:
cd gpt-2

/content/gpt-2


install requirements

In [ ]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 92kB 3.8MB/s 
     |████████████████████████████████| 604kB 6.7MB/s 
     |████████████████████████████████| 61kB 7.9MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 61kB 8.5MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115928 sha256=df38f1c13f88ff9bbd5059a778214bebddf91edb343bca60cd2c9a008a955c18
  Stored in directory: /root/.cache/pip/wheels/af/19/30/1ea0cad502dcb4e66ed5a690279628c827aea38bbbab75d5ed
  Created wheel for regex: filename=regex-2017.4.5-cp37-cp37m-linux_x86_64.whl size=534388 sha256=350a91517e2d6027039bdce9e2a3450af67826e855dba6eddf3d5cedb57293d0
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex
ERROR: spacy 2.2.4 has requirement tqdm<5.0.0,>=4.38.0, but you'll have tqdm 4.31.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but 

download the model

In [ ]:
# download the pretrained model
#!python download_model.py 1558M
#!python download_model.py 117M
# 345M

Fetching checkpoint: 1.00kit [00:00, 1.02Mit/s]                                                     
Fetching encoder.json: 1.04Mit [00:00, 5.63Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 1.07Mit/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:16, 29.6Mit/s]                                  
Fetching model.ckpt.index: 6.00kit [00:00, 5.76Mit/s]                                               
Fetching model.ckpt.meta: 472kit [00:00, 4.23Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 3.20Mit/s]                                                       


In [ ]:
!export PYTHONIOENCODING=UTF-8

In [ ]:
!pip uninstall --yes tensorflow
!pip install tensorflow==1.15 

Uninstalling tensorflow-2.4.1:
  Successfully uninstalled tensorflow-2.4.1
     |████████████████████████████████| 412.3MB 43kB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
     |████████████████████████████████| 3.8MB 32.5MB/s 
     |████████████████████████████████| 512kB 30.4MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=e25e20dbeef59cf6fd9b30816a6450c474e05c79940e58935c8c8f18f2b664f6
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.4.0
    U

In [ ]:
import sys
sys.path.append("/content/gpt-2/src")

In [ ]:
import fire
import json
import os
import numpy as np
import tensorflow as tf

import model, sample, encoder

In [ ]:
import generate_unconditional_samples

In [ ]:
import interactive_conditional_samples

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
class GPT2:

  
  # extracted from the source code to generate some text based on a prior
  def __init__(
      self,
      model_name='117M',
      seed=None,
      nsamples=1,
      batch_size=1,
      length=None,
      temperature=1,
      top_k=0,
      raw_text="",
  ):
      """
      Interactively run the model
      :model_name=117M : String, which model to use
      :seed=None : Integer seed for random number generators, fix seed to reproduce
       results
      :nsamples=1 : Number of samples to return total
      :batch_size=1 : Number of batches (only affects speed/memory).  Must divide nsamples.
      :length=None : Number of tokens in generated text, if None (default), is
       determined by model hyperparameters
      :temperature=1 : Float value controlling randomness in boltzmann
       distribution. Lower temperature results in less random completions. As the
       temperature approaches zero, the model will become deterministic and
       repetitive. Higher temperature results in more random completions.
      :top_k=0 : Integer value controlling diversity. 1 means only 1 word is
       considered for each step (token), resulting in deterministic completions,
       while 40 means 40 words are considered at each step. 0 (default) is a
       special setting meaning no restrictions. 40 generally is a good value.
      """
      if batch_size is None:
          batch_size = 1
      assert nsamples % batch_size == 0

      self.nsamples = nsamples
      self.batch_size = batch_size
      
      my_model_path = "/content/gdrive/MyDrive/Colab Notebooks/NLP/project/"   # your path to the model files
      self.enc = encoder.get_encoder(model_name, my_model_path)  # --------------------------------------
      hparams = model.default_hparams()

      with open(os.path.join(my_model_path, model_name, 'hparams.json')) as f:
          hparams.override_from_dict(json.load(f))

      if length is None:
          length = hparams.n_ctx // 2
      elif length > hparams.n_ctx:
          raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

      self.sess = tf.Session(graph=tf.Graph())
      self.sess.__enter__()
      
      self.context = tf.placeholder(tf.int32, [batch_size, None])
      np.random.seed(seed)
      tf.set_random_seed(seed)
      self.output = sample.sample_sequence(
          hparams=hparams, length=length,
          context=self.context,
          batch_size=batch_size,
          temperature=temperature, top_k=top_k
      )

      saver = tf.train.Saver()
      self.ckpt = tf.train.latest_checkpoint(os.path.join(my_model_path, model_name))

      saver.restore(self.sess, self.ckpt)

  # ei kasutata
  def close(self):
    self.sess.close()
  
  def generate_conditional(self,raw_text):
      context_tokens = self.enc.encode(raw_text)
      for _ in range(self.nsamples // self.batch_size):
          out = self.sess.run(self.output, feed_dict={
              self.context: [context_tokens for _ in range(self.batch_size)]
          })[:, len(context_tokens):]
          for i in range(self.batch_size):
              text = self.enc.decode(out[i])
              return text


In [ ]:
#gpt2 = GPT2(model_name="117M")
gpt2 = GPT2(model_name="117M_81")
# you must also call download_model.py (see earlier cell) with the correct parameter
# 1558M, best results takes a long time to load
# 1558M, 774M, 355M, 345M, 124M, and 117M

Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use `tf.random.categorical` instead.
INFO:tensorflow:Restoring parameters from /content/gdrive/MyDrive/Colab Notebooks/NLP/project/117M_81/model-81756


In [ ]:
# gpt2.close()

In [ ]:
result = gpt2.generate_conditional(raw_text="Can you tell me something about music?")

print(result)


<|endoftext|>
I -- I don't know. Um, I don't <u>know</u> music. Why did you bring me in -- why?
Is there anything you don't know about it?
<|endoftext|>
Please! Please!
'Cause I didn't.
<|endoftext|>
I'm sorry about the sound. Cause that was unreal.
Even we recording people think we're doing this as a joke. But why should we?
<|endoftext|>
No thank you. God we were in chains. You don't hear an earthquake from here? Let's get some air in this wind. These men don't have dif----ed, dif----solution in time. Getting analyzed will help, of course, but what's on my mind --
No, that's fine.
<|endoftext|>
Joel's just doing a job, I understand. And you break off his demo --
So Joel had to do something to his accident?
<|endoftext|>
So Joel had to do something to his accident?
He's the only person I can really <u>stand</u> on. So what?
<|endoftext|>
He's the only person I can really <u>stand</u> on. So what?
He looks like <u>this</u> business?
<|endoftext|>
-- an <u>enlarged</u> heart: -- then <

In [ ]:
class Who:
  """A class defining the conversation parties: me, he"""
  def __init__(self):
    self.prefixes = []

  def matches(self,phrase):
    for prefix in self.prefixes:
      if phrase.startswith(prefix):
        #print(f"{phrase} starts with {prefix}")
        return True
      
    #print(f"{phrase} does not start with {self.prefixes}")
    return False

  def get_random_prefix(self):
    return self.prefixes[0]
  
class Bot(Who):
  def __init__(self):
    super().__init__()
    self.prefixes = ["Bot said: \""]
   
  
class You(Who):
  def __init__(self):
    super().__init__()
    self.prefixes = ["You said: \""]

In [ ]:
class Conversation:
  
  def __init__(self, prior = None):
    if prior is None:
      prior="""
      You said: "Nice to meet you. What's your name?"
      Bot said: "My name is Pete."
      You said: "That's an interesting name. How old are you?"
      Bot said: "I'm 40 years old."
      You said: "Can you tell me something about yourself?"
      Bot said: "Ofcourse! I like playing video games and eating cake. "
      You said: "I like sweet stuff too. What are your plans for tomorrow?"
      """
    self.suggestion = None
    
    self.me = Bot()
    self.you = You()
    self.parties  = [ self.me, self.you ]
    
    self.conversation = []
    
    lines = prior.split("\n")
    for line in lines:
      line = line.strip()
      if len(line)!=0:
        party = None
        for party in self.parties:
          if party.matches(line):
            break
        if party is None:
          raise Exception(f"Unknown party: {line}")
                
        self.conversation.append((party,line))
    self.get_suggestion()
    
  
  def get_prior(self):
    conv = ""
    for (party, line) in self.conversation:
      conv+=line+"\n"
    return conv
  
  def get_suggestion(self):
    who, last_line = self.conversation[-1]

    party_index = self.parties.index(who)
    next_party = self.parties[(party_index+1) % len(self.parties)]
      
    conv = self.get_prior()
    conv += next_party.get_random_prefix()
    answer = self.get_answer(next_party, conv)

    if not next_party.matches(answer):
      prefix = next_party.get_random_prefix()
      answer = prefix + answer
    
    self.suggestion = (next_party, answer)
  
  def next(self, party = None, answer = ""):
    """Continue the conversation
    :param party: None -> use the current party which is currently in turn
    :param answer: None -> use the suggestion, specify a text to override the 
           suggestion
    
    """
    suggested_party, suggested_answer = self.suggestion
    if party is None:
      party = suggested_party
    
    if answer == "":
      answer = suggested_answer
      
    if not party.matches(answer):
      prefix = party.get_random_prefix()
      answer = prefix + answer
    
    answer = answer.strip()
    if answer[-1] != "\"":
      # add the closing "
      answer += "\""
      
    self.conversation.append((party, answer))    
    self.get_suggestion()
    
  def retry(self):
    self.get_suggestion()
        
  def get_answer(self, party, conv):
    answer = gpt2.generate_conditional(raw_text=conv)
    lines = answer.split("\n")
    line = ""
    for line in lines:
      if line !="":
        break
      
    if line!="":
      return line
    
    return ""
      
  def show(self):
    conv = ""
    for (party, line) in self.conversation:
      conv+=line+"\n"
    print(conv)
    if self.suggestion is not None:
      party, answer  = self.suggestion
      print("--> "+answer)
    
    


In [ ]:
#from IPython.display import clear_output

In [ ]:
#c = Conversation() 

def CLI():
  #######################################
  c = Conversation()  # kommenteeri välja kui ei taha iga kord dialoogi uuesti algusest alustada
  #######################################
  cmdl = ''
  while True:
    print("-------------------------------------\nPrior conversation:\n")
    c.show()
    print("\nAre you satisfied with the generated answer? (y/n) ('y' means you can continue on with this answer, 'n' means a new response is generated) Type 'exit' to quit.")
    cmdl = str(input(">>>"))
    cmdl = cmdl.lower()
    cmdl = cmdl.strip()
    if cmdl == "exit":
      print("Bye!")
      break
    elif cmdl == "n":
      #clear_output()
      print("Generating new response. Please wait a bit.")
      c.retry()
    elif cmdl == "y":
      #clear_output()
      c.next() # accept the answer
      print("Please give your reply:")
      cmdl = str(input(">>>"))
      cmdl = cmdl.lower()
      cmdl = cmdl.strip()
      c.next(c.you, cmdl)
    else:
      print("This is not valid input!")

In [ ]:
CLI()

-------------------------------------
Prior conversation:

You said: "Nice to meet you. What's your name?"
Bot said: "My name is Pete."
You said: "That's an interesting name. How old are you?"
Bot said: "I'm 40 years old."
You said: "Can you tell me something about yourself?"
Bot said: "Ofcourse! I like playing video games and eating cake. "
You said: "I like sweet stuff too. What are your plans for tomorrow?"

--> Bot said: "I'm working on a video game. I would love to play the new Nintendo game. It's crazy fun. "

Are you satisfied with the generated answer? (y/n) ('y' means you can continue on with this answer, 'n' means a new response is generated) Type 'exit' to quit.
>>>y
Please give your reply:
>>>so you plan on playing games all day?
-------------------------------------
Prior conversation:

You said: "Nice to meet you. What's your name?"
Bot said: "My name is Pete."
You said: "That's an interesting name. How old are you?"
Bot said: "I'm 40 years old."
You said: "Can you tell m